In [6]:
from queue import Queue
from collections import deque

class MCProblem:
    class State:
        def __init__(self, missionaries, cannibals, boat):
            # initializes the state with the number of missionaries, cannibals and boat position
            self.missionaries = missionaries
            self.cannibals = cannibals
            self.boat = boat
            self.parent = None  # set parent to None-+
        
        def __eq__(self, other):
            # checks if two states are equal
            return self.missionaries == other.missionaries and self.cannibals == other.cannibals and self.boat == other.boat
    
        def __hash__(self):
            # hashes the state tuple
            return hash((self.missionaries, self.cannibals, self.boat))
    
        def __str__(self):
            # string representation of the state
            return "M: {} C: {} Boat: {}".format(self.missionaries, self.cannibals, self.boat)

    def __init__(self):
        # initializes the initial state with 3 missionaries, 3 cannibals and the boat on the starting side
        self.initial_state = self.State(3, 3, 1)

    def actions(self, state):
        # returns a list of valid actions for a given state
        if state.boat == 1:
            actions = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]
        else:
            actions = [(-1, 0), (-2, 0), (0, -1), (0, -2), (-1, -1)]
        
        valid_actions = []
        for action in actions:
            if self.is_valid(state, action):
                valid_actions.append(action)
                
        return valid_actions
    
    def is_valid(self, state, action):
        # checks if the given action is valid for the given state
        missionaries = state.missionaries - action[0] * state.boat
        cannibals = state.cannibals - action[1] * state.boat
        
        if missionaries < 0 or cannibals < 0 or missionaries > 3 or cannibals > 3:
            return False
        
        if (missionaries != 0 and missionaries < cannibals) or (3 - missionaries != 0 and 3 - missionaries < 3 - cannibals):
            return False
        
        return True

    def result(self, state, action):
        # returns the new state resulting from the given action on the given state
        new_state = self.State(state.missionaries - action[0] * state.boat, state.cannibals - action[1] * state.boat, 1 - state.boat)
        new_state.parent = state  # set the parent of the new state to the given state
        new_state.action = action  # store the action that led to this state
        return new_state

    def goal_test(self, state):
        # checks if the given state is the goal state
        return state.missionaries == 0 and state.cannibals == 0 and state.boat == 0

    def bfs_search(self):
        start_node = self.State(3, 3, 1)
        if self.goal_test(start_node):
            return [start_node]

        frontier = deque([start_node])  # initialize the frontier with the start node
        explored = set()  # initialize the explored set frontier:
        
        while frontier: # loop until the frontier is empty
            node = frontier.popleft() # get the leftmost node from the frontier
            explored.add(node)  # add the node to the set of explored nodes

        #Generate all possible actions that can be taken from the current node
            for action in self.actions(node):
                child_state = self.result(node, action)
                if child_state not in explored:
                    child_node = self.State(child_state.missionaries, child_state.cannibals, child_state.boat)
                    child_node.parent = node
                    child_node.action = action
                    if self.goal_test(child_state):  # If the child node is the goal state, return the solution path
                        path = []
                        while child_node:
                            path.append(child_node)
                            child_node = child_node.parent
                        return list(reversed(path))
                    frontier.append(child_node)

        return None
# Create an instance of the MCProblem class    
problem = MCProblem()

# Use the bfs_search method to find a solution path
solution_path = problem.bfs_search()
if solution_path:
    for state in solution_path:
        print(state)
else:
    print("No solution found.")
         

M: 3 C: 3 Boat: 1
M: 3 C: 1 Boat: 0
M: 3 C: 1 Boat: 1
M: 1 C: 1 Boat: 0
M: 1 C: 1 Boat: 1
M: 0 C: 0 Boat: 0
